In [5]:
# workhorse modules
import pandas as pd
from datetime import timedelta, datetime
import re
from pathlib import Path
import seaborn as sns
import matplotlib.pyplot as plt
import ipywidgets as widgets

# local utility functions
from utils.general_utils import (
    add_fiscal_year,
    compute_is_on_time,
    set_pd_params,
    tidy_up_df,
    cast_dtypes,
    compute_pm_cm,
    compute_pm_cm_by_month,
    compute_kpi_table,
    compute_kpi_table_by_month,
)
from utils.vis_utils import set_plot_params, pointplot_with_barplot
from utils.anonymization import split_names, replace_names

In [2]:
import sys
sys.version

'3.8.5 (default, Apr 15 2021, 10:03:23) \n[Clang 10.0.1 (clang-1001.0.46.4)]'

In [4]:
scorecard_path = Path.cwd() / 'data' / 'scorecard_data.pkl'
data = pd.read_pickle(scorecard_path)
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 108105 entries, 0 to 108104
Data columns (total 18 columns):
 #   Column                  Non-Null Count   Dtype         
---  ------                  --------------   -----         
 0   wr_id                   108105 non-null  int64         
 1   status                  108105 non-null  object        
 2   date_completed          98010 non-null   datetime64[ns]
 3   date_requested          108105 non-null  datetime64[ns]
 4   date_closed             98966 non-null   datetime64[ns]
 5   fy_request              108105 non-null  int64         
 6   fy_close                98966 non-null   float64       
 7   role_name               100612 non-null  object        
 8   building_name           107798 non-null  object        
 9   b_number                107798 non-null  object        
 10  primary_type            108105 non-null  object        
 11  problem_type            108105 non-null  object        
 12  pm_group                8718 n

In [6]:
def say_my_name(name):
    """
    Print the current widget value in short sentence
    """
    print(f'My name is {name}')
     
widgets.interact(say_my_name, name=["Jim", "Emma", "Bond"]);

interactive(children=(Dropdown(description='name', options=('Jim', 'Emma', 'Bond'), value='Jim'), Output()), _…